In [19]:
import pandas as pd
import numpy as np

from tqdm._tqdm_notebook import tqdm_notebook
# If error -> pip install tqdm

#### 1. Data import

In [20]:
conflict_df = pd.read_csv('datasets/ucdp-prio-acd-171.csv')
conflict_df.head()

,conflictid,location,sidea,sidea2nd,side b,sidebid,sideb2nd,incomp,terr,year,...,epend,ependdate,ependprec,gwnoa,gwnoa2nd,gwnob,gwnob2nd,gwnoloc,region,version
0,200,Bolivia,Government of Bolivia,NaN,Popular Revolutionary Movement,719,NaN,2,NaN,1946,...,1,1946-07-21,1.0,145,NaN,NaN,NaN,145,5,17.1
1,200,Bolivia,Government of Bolivia,NaN,MNR,720,NaN,2,NaN,1952,...,1,1952-04-12,1.0,145,NaN,NaN,NaN,145,5,17.1
2,200,Bolivia,Government of Bolivia,NaN,ELN,721,NaN,2,NaN,1967,...,1,1967-10-16,1.0,145,NaN,NaN,NaN,145,5,17.1
3,201,Cambodia (Kampuchea),Government of France,NaN,Khmer Issarak,160,NaN,1,Cambodia,1946,...,0,NaN,NaN,220,NaN,NaN,NaN,811,3,17.1
4,201,Cambodia (Kampuchea),Government of France,NaN,Khmer Issarak,160,NaN,1,Cambodia,1947,...,0,NaN,NaN,220,NaN,NaN,NaN,811,3,17.1


#### 2. Dataframe process

- remove useless columns
- remove all "Government of..."
- remove text between parantheses
- complete/correct "terr" columns

In [21]:
# Remove useless columns
columns_to_drop = ['conflictid', 'sidebid', 'incomp', 'terr', 'startprec', 'sidea2nd',
                   'startprec2', 'epend', 'ependprec', 'gwnoa', 'gwnoa2nd', 'sideb2nd',
                   'gwnob', 'gwnob2nd', 'gwnoloc', 'version']

conflict_df.drop(columns_to_drop, axis=1, inplace=True)

# Remove columns
conflict_df.replace({'Government of ': ''}, regex=True, inplace=True)
# Remove text between parentheses 
conflict_df.location.replace({r'\([^()]+\)' : ''}, regex=True, inplace=True)
# Remove South/North from Location (ex: South Yemen or South Korea)
conflict_df.location.replace({'South ' : ''}, regex=True, inplace=True)
conflict_df.location.replace({'North ' : ''}, regex=True, inplace=True)
# But we keep South Africa
conflict_df.location.replace({'Africa' : 'South Africa'}, regex=True, inplace=True)
conflict_df.location.replace({'Central South African Republic' : 'Central African Republic'}, regex=True, inplace=True)

conflict_df.location.replace({'Congo' : 'DR Congo'}, regex=True, inplace=True)
conflict_df.location.replace({'DR DR' : 'DR'}, regex=True, inplace=True)


conflict_df.head()

,location,sidea,side b,year,intensity,cumint,type,startdate,startdate2,ependdate,region
0,Bolivia,Bolivia,Popular Revolutionary Movement,1946,2,1,3,1946-07-18,1946-07-21,1946-07-21,5
1,Bolivia,Bolivia,MNR,1952,1,1,3,1946-07-18,1952-04-09,1952-04-12,5
2,Bolivia,Bolivia,ELN,1967,1,1,3,1946-07-18,1967-03-31,1967-10-16,5
3,Cambodia,France,Khmer Issarak,1946,1,0,1,1946-08-31,1946-08-31,NaN,3
4,Cambodia,France,Khmer Issarak,1947,1,0,1,1946-08-31,1946-08-31,NaN,3


In [44]:
columns = ["location", "ID", "Colonizer Country", "Indep Date", "sidea", "side b", "incomp", "year", "intensity", 
               "cumint", "type", "startdate", "startdate2", "ependdate", "region"]

conflict_df_tmp = pd.DataFrame(columns=columns)

for index in range(len(conflict_df)):

    locations = conflict_df.loc[index, 'location'].split(",")
    regions = conflict_df.loc[index, 'region'].split(",")
    
    if len(regions) > 1:
        
        pairs = [(locations[i], regions[i]) for i in range(len(locations))]

        for i, pair in enumerate(pairs):
            
            # Get information from colonies dataframe
            indep_year = colonized_df.loc[colonized_df['Colonized Country'] == country, 'Year'].values[0]

            
            #int(colonized_df[colonized_df['Colonized Country']==country]['Year'])
            indep_month = int(colonized_df[colonized_df['Colonized Country']==country]['Month'])
            indep_day = int(colonized_df[colonized_df['Colonized Country']==country]['Day'])
            indep_date = str(indep_day) + "/" + str(indep_month) + "/" + str(indep_year)
        
            ID = list(colonized_df[colonized_df['Colonized Country']==country]['ID'])[0]

            start_year = int(conflict_df.loc[index, 'startdate'].split("-")[0])

            conflict_df_tmp = conflict_df.loc[index:index,]
            print(conflict_df_tmp)
            conflict_df_tmp.set_value(index, 'Indep Date', indep_date)
            conflict_df_tmp.set_value(index, 'ID', ID)
            
            
            colonizer_country = colonized_df.loc[colonized_df['Colonized Country'] == country, 'Colonizer Country'].values[0]

            
            #colonizer_country = colonized_df.get_value[colonized_df['Colonized Country'] == country, 'country']
            print("index:", index, "lol",colonizer_country)
            print(indep_year, type(indep_year))
            conflict_df_tmp.set_value(index, "Colonizer Country", colonizer_country)
                
            # Add the new line to the df
            conflict_df_tmp = conflict_df_tmp.append(conflict_row_df)
        
        # Remove initial line
        conflict_df.drop(index, inplace=True)#.reset_index(drop=True)
                  
    elif len(locations) > 1:
        conflict_df.loc[index, 'location'] = locations[0]
                
            
#test      
conflict_df = pd.concat([conflict_df,conflict_df_tmp]).reset_index(drop=True)
conflict_df = conflict_df[columns]

index: 2277 lol United Kingdom
1776 <class 'numpy.int64'>
index: 2277 lol United Kingdom
1776 <class 'numpy.int64'>
index: 2277 lol United Kingdom
1776 <class 'numpy.int64'>


#### 3. Keep ex-colonies


In [5]:
colonized_df = pd.read_csv('datasets/colonies_wikipedia.csv')
colonized_countries = list(colonized_df["Colonized Country"])
colonized_df.head()

,Unnamed: 0,Colonized Country,ID,Day,Month,Year,Colonizer Country,URL
0,1,Iceland,IS,17,6,1944,Denmark,/wiki/Iceland
1,2,Canada,CA,1,7,1867,United Kingdom,/wiki/Canada
2,3,United States of America,US,4,7,1776,United Kingdom,/wiki/United_States
3,4,Haiti,HT,1,1,1804,France,/wiki/Haiti
4,5,Benin,BJ,1,8,1960,France,/wiki/Benin


In [ ]:
columns = ["location", "ID", "sidea", "side b", "year", "intensity", "cumint", "type", 
           "startdate", "startdate2", "ependdate", "region"]

pre_colonization_conflict_df = pd.DataFrame(columns=columns)
post_colonization_conflict_df = pd.DataFrame(columns=columns)

for index in tqdm_notebook(range(len(conflict_df))):
    country = conflict_df.loc[index, 'location']
    
    if country[-1] == ' ': country = country[:-1]
    
    if country in colonized_countries or country + " " in colonized_countries:
        indep_year = int(colonized_df[colonized_df['Colonized Country']==country]['Year'])
        indep_month = int(colonized_df[colonized_df['Colonized Country']==country]['Month'])
        indep_day = int(colonized_df[colonized_df['Colonized Country']==country]['Day'])
        indep_date = str(indep_day) + "/" + str(indep_month) + "/" + str(indep_year)
        
        start_year = int(conflict_df.loc[index, 'startdate'].split("-")[0])

        print(indep_date, type(indep_year ))
        conflict_df_tmp = conflict_df.loc[index:index,]
        ID = list(colonized_df[colonized_df['Colonized Country']==country]['ID'])[0]
        conflict_df_tmp.set_value(index, 'ID', ID)
        
        if start_year > indep_year:
            post_colonization_conflict_df = post_colonization_conflict_df.append(conflict_df_tmp)
        else:
            pre_colonization_conflict_df = pre_colonization_conflict_df.append(conflict_df_tmp)
    #else:
    #    print(country, "--> NOT EX COLONY")
        
post_colonization_conflict_df.reset_index(drop=True)
post_colonization_conflict_df = post_colonization_conflict_df[columns]
post_colonization_conflict_df.to_csv("datasets/post_colonization_conflict.csv")

pre_colonization_conflict_df.reset_index(drop=True)
pre_colonization_conflict_df = pre_colonization_conflict_df[columns]
post_colonization_conflict_df.to_csv("datasets/pre_colonization_conflict.csv")

Widget Javascript not detected.  It may not be installed or enabled properly.


1825 <class 'int'>


/home/jean/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:337: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
/home/jean/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


1825 <class 'int'>
1825 <class 'int'>
1953 <class 'int'>
1953 <class 'int'>
1953 <class 'int'>
1953 <class 'int'>
1953 <class 'int'>
1953 <class 'int'>
1953 <class 'int'>
1953 <class 'int'>
1945 <class 'int'>
1945 <class 'int'>
1945 <class 'int'>
1945 <class 'int'>
1948 <class 'int'>
1949 <class 'int'>
1949 <class 'int'>
1949 <class 'int'>
1949 <class 'int'>
1949 <class 'int'>
1949 <class 'int'>
1949 <class 'int'>
1949 <class 'int'>
1898 <class 'int'>
1898 <class 'int'>
1898 <class 'int'>
1898 <class 'int'>
1898 <class 'int'>
1898 <class 'int'>
1898 <class 'int'>
1898 <class 'int'>
1898 <class 'int'>
1898 <class 'int'>
1898 <class 'int'>
1898 <class 'int'>
1898 <class 'int'>
1898 <class 'int'>
1898 <class 'int'>
1898 <class 'int'>
1898 <class 'int'>
1898 <class 'int'>
1898 <class 'int'>
1898 <class 'int'>
1898 <class 'int'>
1898 <class 'int'>
1898 <class 'int'>
1898 <class 'int'>
1898 <class 'int'>
1898 <class 'int'>
1898 <class 'int'>
1898 <class 'int'>
1898 <class 'int'>
1898 <class 

Exception in thread Thread-12:
Traceback (most recent call last):
  File "/home/jean/anaconda3/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/home/jean/anaconda3/lib/python3.6/site-packages/tqdm/_tqdm.py", line 144, in run
    for instance in self.tqdm_cls._instances:
  File "/home/jean/anaconda3/lib/python3.6/_weakrefset.py", line 60, in __iter__
    for itemref in self.data:
RuntimeError: Set changed size during iteration



 <class 'int'>
1947 <class 'int'>
1947 <class 'int'>
1947 <class 'int'>
1947 <class 'int'>
1947 <class 'int'>
1947 <class 'int'>
1947 <class 'int'>
1947 <class 'int'>
1947 <class 'int'>
1947
 <class 'int'>
1960 <class 'int'>
1811 <class 'int'>
1811 <class 'int'>
1811 <class 'int'>
1948 <class 'int'>
1948 <class 'int'>
1948 <class 'int'>
1948 <class 'int'>
1948 <class 'int'>
1948 <class 'int'>
1948 <class 'int'>
1948 <class 'int'>
1948 <class 'int'>
1948 <class 'int'>
1948 <class 'int'>
1948 <class 'int'>
1948 <class 'int'>
1948 <class 'int'>
1948 <class 'int'>
1948 <class 'int'>
1948 <class 'int'>
1948 <class 'int'>
1948 <class 'int'>
1948 <class 'int'>
1948 <class 'int'>
1948 <class 'int'>
1948 <class 'int'>
1948 <class 'int'>
1948 <class 'int'>
1948 <class 'int'>
1948 <class 'int'>
1948 <class 'int'>
1948 <class 'int'>
1948 <class 'int'>
1948 <class 'int'>
1948 <class 'int'>
1948 <class 'int'>
1948 <class 'int'>
1948 <class 'int'>
1948 <class 'int'>
1948 <class 'int'>
1948 <class 'in

1941 <class 'int'>
1941 <class 'int'>
1941 <class 'int'>
1941 <class 'int'>
1811 <class 'int'>
1811 <class 'int'>
1811 <class 'int'>
1962 <class 'int'>
1973 <class 'int'>
1973 <class 'int'>
1973 <class 'int'>
1973 <class 'int'>
1973 <class 'int'>
1973 <class 'int'>
1973 <class 'int'>
1973 <class 'int'>
1973 <class 'int'>
1973 <class 'int'>
1973 <class 'int'>
1957 <class 'int'>
1957 <class 'int'>
1957 <class 'int'>
1957 <class 'int'>
1945 <class 'int'>
1945 <class 'int'>
1945 <class 'int'>
1945 <class 'int'>
1956 <class 'int'>
1956 <class 'int'>
1956 <class 'int'>
1956 <class 'int'>
1956 <class 'int'>
1956 <class 'int'>
1956 <class 'int'>
1956 <class 'int'>
1956 <class 'int'>
1956 <class 'int'>
1960 <class 'int'>
1960 <class 'int'>
1960 <class 'int'>
1960 <class 'int'>
1960 <class 'int'>
1960 <class 'int'>
1960 <class 'int'>
1960 <class 'int'>
1960 <class 'int'>
1960 <class 'int'>
1960 <class 'int'>
1960 <class 'int'>
1960 <class 'int'>
1960 <class 'int'>
1960 <class 'int'>
1960 <class 

1947 <class 'int'>
1947 <class 'int'>
1947 <class 'int'>
1947 <class 'int'>
1821 <class 'int'>
1821 <class 'int'>
1821 <class 'int'>
1821 <class 'int'>
1821 <class 'int'>
1821 <class 'int'>
1821 <class 'int'>
1821 <class 'int'>
1821 <class 'int'>
1821 <class 'int'>
1821 <class 'int'>
1821 <class 'int'>
1960 <class 'int'>
1960 <class 'int'>
1960 <class 'int'>
1960 <class 'int'>
1960 <class 'int'>
1960 <class 'int'>
1960 <class 'int'>
1960 <class 'int'>
1960 <class 'int'>
1960 <class 'int'>
1960 <class 'int'>
1960 <class 'int'>
1960 <class 'int'>
1960 <class 'int'>
1960 <class 'int'>
1960 <class 'int'>
1960 <class 'int'>
1960 <class 'int'>
1960 <class 'int'>
1960 <class 'int'>
1960 <class 'int'>
1960 <class 'int'>
1960 <class 'int'>
1960 <class 'int'>
1960 <class 'int'>
1960 <class 'int'>
1960 <class 'int'>
1956 <class 'int'>
1965 <class 'int'>
1910 <class 'int'>
1910 <class 'int'>
1910 <class 'int'>
1910 <class 'int'>
1910 <class 'int'>
1910 <class 'int'>
1910 <class 'int'>
1947 <class 

In [ ]:
print("Conflict pre coloinzation :", len(pre_colonization_conflict_df))
print("Conflict post coloinzation :", len(post_colonization_conflict_df))